라이브러리

In [ ]:
import numpy as np
import pickle
from tensorflow.keras.datasets import mnist

1. MNIST 데이터셋
    - MNIST는 0부터 9까지 숫자 손글씨 이미지로 구성된 데이터셋이다.
        - 입력 데이터 | 28×28 크기의 회색조 이미지 (1채널)
        - 훈련 데이터 | 60,000장
        - 시험 데이터 | 10,000장
        - 레이블(Label) | 0~9 중 하나 (정답 숫자)

2. MNIST 데이터 불러오기

In [3]:
from tensorflow.keras.datasets import mnist

# 데이터 다운로드 및 로드
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# 데이터 형태 출력
print(x_train.shape)  # (60000, 28, 28)
print(t_train.shape)  # (60000,)
print(x_test.shape)   # (10000, 28, 28)
print(t_test.shape)   # (10000,)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


    - 데이터 전처리 옵션
        - flatten=True : 이미지를 1차원 배열(784차원)로 변환
        - normalize=True : 픽셀값을 0.0~1.0 범위로 정규화
        - one_hot_label=True : 레이블을 원-핫 인코딩 형식으로 변환

3. 신경망 구조
    - 입력층: 784개 (28×28=784)
    - 첫 번째 은닉층: 50개 뉴런
    - 두 번째 은닉층: 100개 뉴런
    - 출력층: 10개 뉴런 (0~9 분류)

4. 신경망 초기화 및 추론 함수

- 4.1 데이터 가져오기

In [6]:
def get_data():
    # 데이터 로드
    (x_train, t_train), (x_test, t_test) = mnist.load_data()
    # flatten (28x28 → 784)
    x_test = x_test.reshape(x_test.shape[0], -1)
    # normalize (0~255 → 0~1)
    x_test = x_test.astype('float32') / 255.0
    return x_test, t_test

- 4.2 학습된 네트워크 불러오기

In [7]:
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

    - sample_weight.pkl 파일에는 학습된 가중치(W)와 편향(b) 값이 저장되어 있다.
    - 이 파일을 읽어와서 학습 없이 바로 추론(inference) 가능.

- 4.3 추론(Forward) 함수

In [16]:
# 올바른 순서
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)  # 오버플로 방지용 상수
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)

    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)

    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


    - 각 층마다 선형변환 → 활성화 함수(sigmoid) 적용
    - 마지막 층은 softmax로 확률 변환

5. 정확도 측정

- 5.1 기본 방식 (1개씩 처리)

In [17]:
accuracy_cnt = 0

x, t = get_data()
network = init_network()

for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 확률이 가장 높은 클래스 선택
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:", str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


    - 1개씩 예측해서 맞춘 개수 세기
    - Accuracy ≈ 0.9352 (약 93.5%)

- 5.2 배치 처리 (속도 향상)

In [18]:
batch_size = 100  # 배치 크기

accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:", str(float(accuracy_cnt) / len(x)))


Accuracy: 0.9352
